In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import sys
from loguru import logger
logger.remove()
logger.add(sys.stdout, level="INFO")

1

In [19]:
logger.remove()
logger.add(sys.stdout, level="DEBUG")

2

In [3]:
import sys
import os
sys.path.insert(0, '../')

from llmagent.privatestore import WebStore, WebConf
from llmagent.secret import AK_SK
from llmagent.llmapi import QianfanLLM, QianfanEmbedding, model_spec
keys = AK_SK('../llmagent/secret/keystore/qianfan.keys')
os.environ["QIANFAN_ACCESS_KEY"] = keys.get_ak()
os.environ["QIANFAN_SECRET_KEY"] = keys.get_sk()

llm = QianfanLLM(model_spec=model_spec.Speed128K, temperature=0.6)
embedding = QianfanEmbedding()
webstore = WebStore("llmtutorial", "../vectorstore/llmtutorial", emb_func=embedding, extra_config=WebConf(
    root_url="https://aitutor.liduos.com/",
    limit=10,
))

USER_AGENT environment variable not set, consider setting it to identify your requests.
[WARNING][2024-10-24 08:49:24.193] redis_rate_limiter.py:21 [t:8703163072]: no redis installed, RedisRateLimiter unavailable


In [3]:
# webstore.reload()

In [5]:
webstore.collection_count()

2234

In [4]:
webstore._documents['https://aitutor.liduos.com/01-llm/01-2.html'].page_content

'Title    你好, ChatGPT (..)         本电子书开源，欢迎 star 🌟，关注《LLM 应用开发实践笔记》  我的新书 《LangChain编程从入门到实践》 (https://u.jd.com/V8pkqFY) 已经开售！推荐正在学习AI应用开发的朋友购买阅读！   你好, ChatGPT  ChatGPT 是OpenAI开发的人工智能聊天机器人程序，于2022年11月推出。该程序使用基于 GPT-3.5、GPT-4 架构的大语言模型并以强化学习训练。ChatGPT目前仍以文字方式交互，而除了可以用人类自然对话方式来交互，还可以用于甚为复杂的语言工作，包括自动生成文本、自动问答、自动摘要等多种任务。  ChatGPT的诞生  演进过程     总结    训练  训练有四个主要阶段：预训练、有监督微调、奖励建模、强化学习  Pretraining 预训练    Supervised Finetuning 监督微调    Reward Modeling 奖励建模    Reinforcement Learning 强化学习    特点    局限    参考链接        results matching " "     No results matching " "         (01-1.html)  (01-1.html)   (01-3.html)  (01-3.html)'

## 构建Prompt

In [27]:
from llmagent.chain import QAWithContextChain, TemplateConf
prompt = QAWithContextChain(llm=llm, 
                               vectorstore=webstore.vectorstore(),
                               conf=TemplateConf(context_num=10, history_num=3))

In [29]:
import time
ans = prompt.stream("langchain提示词管理")
for token in ans:
    for c in token:
        time.sleep(0.1)
        print(c, end="")
    

2024-10-24 09:05:36.826 | DEBUG    | llmagent.llmapi.llm:_stream:38 - Prompt: 你是一个解答用户问题的assistant，可以根据context资料回答问题。请尽量保证回答的内容都可以在context中找到根据，并务必保留资料最后的 source。以下是context资料：
Title    Agent 介绍 (..)         本电子书开源，欢迎 star 🌟，关注《LLM 应用开发实践笔记》  我的新书 《LangChain编程从入门到实践》 (https://u.jd.com/V8pkqFY) 已经开售！推荐正在学习AI应用开发的朋友购买阅读！   什么是 AI 代理  AI代理（AI agent）是指使用 AI
source: https://aitutor.liduos.com/07-agents/07-1.html

Title    Agent 介绍 (..)         本电子书开源，欢迎 star 🌟，关注《LLM 应用开发实践笔记》  我的新书 《LangChain编程从入门到实践》 (https://u.jd.com/V8pkqFY) 已经开售！推荐正在学习AI应用开发的朋友购买阅读！   什么是 AI 代理  AI代理（AI agent）是指使用 AI
source: https://aitutor.liduos.com/07-agents/07-1.html

Title    Agent 介绍 (..)         本电子书开源，欢迎 star 🌟，关注《LLM 应用开发实践笔记》  我的新书 《LangChain编程从入门到实践》 (https://u.jd.com/V8pkqFY) 已经开售！推荐正在学习AI应用开发的朋友购买阅读！   什么是 AI 代理  AI代理（AI agent）是指使用 AI
source: https://aitutor.liduos.com/07-agents/07-1.html

Title    LLM 安全专题 (..)         本电子书开源，欢迎 star 🌟，关注《LLM 应用开发实践笔记》  我的新书 《LangChain编程从入门到实践》 (https://u.jd.com/V8pkqFY) 已

[WARNING][2024-10-24 09:05:36.828] base.py:916 [t:8703163072]: This key `disable_search` does not seem to be a parameter that the model `ERNIE-Speed-128K` will accept


langchain提示词管理是指在使用大型语言模型（LLM）进行应用开发时，对于提供给模型的输入文本（即提示词）的管理。提示词是引导大型语言模型生成特定主题或类型文本的关键。在langchain框架中，提示词管理是一个重要的部分，它涉及到提示词的编排、日志监控、测试及分析等工作。具体来说，langchain提示词管理可能包括以下几个方面：

1. 提示词编排：根据需求，将提示词进行组织、归类和格式化，以便更好地引导模型生成所需的输出。2. 日志监控：对提示词的使用情况进行监控，以便了解哪些提示词更有效，哪些可能需要调整。3. 测试与分析：通过测试不同的提示词，分析它们对模型输出的影响，从而优化提示词的效果。在langchain中，提示词管理与模型服务层通过API相关联，这有助于更高效地与大型语言模型进行交互。参考资料：

* https://aitutor.liduos.com/06-llmops/06-3.html
* https://aitutor.liduos.com/02-langchain/02-2-1.html

希望以上内容对你有帮助，更多关于langchain提示词管理的信息，建议咨询相关领域的专业人士或查阅langchain官方文档。

In [30]:
ans = prompt.stream("langchain回调管理器如何使用")
for token in ans:
    for c in token:
        time.sleep(0.03)
        print(c, end="")

2024-10-24 09:12:30.621 | DEBUG    | llmagent.llmapi.llm:_stream:38 - Prompt: 你是一个解答用户问题的assistant，可以根据context资料回答问题。请尽量保证回答的内容都可以在context中找到根据，并务必保留资料最后的 source。以下是context资料：
LangChain模块之 Callbacks (https://python.langchain.com/docs/modules/callbacks/)  回调模块允许接到LLM应用程序的各个阶段，鉴于LLM的幻觉问题，这对于日志记录、监视、流式处理和其他任务非常有用，现在也有专用的工具Helicone，Arize AI等产品可用，具体看 LLM应用生态初创公司说明 (../ref/company.md)  自定义回调对象  所有的回调对象都是基于这个基类来声明的  class  BaseCallbackHandler :  """Base callback handler that
source: https://aitutor.liduos.com/02-langchain/02-2-3.html

LangChain模块之 Callbacks (https://python.langchain.com/docs/modules/callbacks/)  回调模块允许接到LLM应用程序的各个阶段，鉴于LLM的幻觉问题，这对于日志记录、监视、流式处理和其他任务非常有用，现在也有专用的工具Helicone，Arize AI等产品可用，具体看 LLM应用生态初创公司说明 (../ref/company.md)  自定义回调对象  所有的回调对象都是基于这个基类来声明的  class  BaseCallbackHandler :  """Base callback handler that
source: https://aitutor.liduos.com/02-langchain/02-2-3.html

LangChain模块之 Callbacks (https://python.langchain.com/docs/modules/callbacks/)  回调模块允许接到LLM应用程序的各个阶段，鉴于LLM的幻觉

[WARNING][2024-10-24 09:12:30.622] base.py:916 [t:8703163072]: This key `disable_search` does not seem to be a parameter that the model `ERNIE-Speed-128K` will accept


在LangChain中，回调管理器是一个非常有用的模块，它允许开发者在大型语言模型（LLM）应用程序的各个阶段插入自定义回调函数，以实现日志记录、监视、流式处理等其他任务。为了使用LangChain的回调管理器，你可以遵循以下步骤：

1. 自定义回调对象：所有的回调对象都是基于`BaseCallbackHandler`这个基类来声明的。你可以根据自己的需求继承这个基类并实现特定的功能。2. 注册回调：在LangChain应用程序中，你需要将自定义的回调对象注册到回调管理器中。这样，当应用程序的特定阶段触发时，相应的回调函数就会被调用。3. 实现回调函数：在自定义的回调对象中，你需要实现具体的回调函数。这些函数将根据应用程序的阶段和事件进行触发，并执行相应的任务，如日志记录、数据监视等。4. 配置和使用：根据LangChain的文档和示例，配置回调管理器的使用。这可能涉及到在配置文件中指定回调对象的参数，或者在代码中直接实例化并配置回调对象。请注意，由于LangChain是一个不断发展和迭代的框架，具体的API和使用方式可能会有所变化。为了获得最准确和最新的信息，建议查阅LangChain的官方文档、参考示例代码或咨询相关领域的专业人士。此外，对于大型语言模型（LLM）的幻觉问题，回调管理器也可以用于日志记录和监视，以帮助识别和减少模型的幻觉。这也是使用LangChain回调管理器的一个重要应用场景。希望以上信息对你有所帮助！如果你还有其他问题，欢迎继续提问。

In [38]:
prompt.conf.history_num=2

In [35]:
prompt._get_history()

[HumanMessage(content='langchain回调管理器如何使用', additional_kwargs={}, response_metadata={}),
 AIMessage(content='在LangChain中，回调管理器是一个非常有用的模块，它允许开发者在大型语言模型（LLM）应用程序的各个阶段插入自定义回调函数，以实现日志记录、监视、流式处理等其他任务。为了使用LangChain的回调管理器，你可以遵循以下步骤：\n\n1. 自定义回调对象：所有的回调对象都是基于`BaseCallbackHandler`这个基类来声明的。你可以根据自己的需求继承这个基类并实现特定的功能。2. 注册回调：在LangChain应用程序中，你需要将自定义的回调对象注册到回调管理器中。这样，当应用程序的特定阶段触发时，相应的回调函数就会被调用。3. 实现回调函数：在自定义的回调对象中，你需要实现具体的回调函数。这些函数将根据应用程序的阶段和事件进行触发，并执行相应的任务，如日志记录、数据监视等。4. 配置和使用：根据LangChain的文档和示例，配置回调管理器的使用。这可能涉及到在配置文件中指定回调对象的参数，或者在代码中直接实例化并配置回调对象。请注意，由于LangChain是一个不断发展和迭代的框架，具体的API和使用方式可能会有所变化。为了获得最准确和最新的信息，建议查阅LangChain的官方文档、参考示例代码或咨询相关领域的专业人士。此外，对于大型语言模型（LLM）的幻觉问题，回调管理器也可以用于日志记录和监视，以帮助识别和减少模型的幻觉。这也是使用LangChain回调管理器的一个重要应用场景。希望以上信息对你有所帮助！如果你还有其他问题，欢迎继续提问。', additional_kwargs={}, response_metadata={})]

In [36]:
ans = prompt.stream("除了langchain，还有哪些ChatGPT框架")
for token in ans:
    for c in token:
        time.sleep(0.03)
        print(c, end="")

2024-10-24 09:13:24.433 | DEBUG    | llmagent.llmapi.llm:_stream:38 - Prompt: 你是一个解答用户问题的assistant，可以根据context资料回答问题。请尽量保证回答的内容都可以在context中找到根据，并务必保留资料最后的 source。以下是context资料：
the ChatGPT API》教程面向想要基于 LLM 开发应用程序的开发者，简洁有效而又系统全面地介绍了如何基于 ChatGPT API 打造完整的对话系统；《LangChain for LLM Application Development》教程结合经典大模型开源框架 LangChain，介绍了如何基于 LangChain 框架开发具备实用功能、能力全面的应用程序，《LangChain Chat With Your Data》教程则在此基础上进一步介绍了如何使用 LangChain 架构结合个人私有数据开发个性化大模型应用；《Building Generative AI
source: https://aitutor.liduos.com/ref/ref.html

the ChatGPT API》教程面向想要基于 LLM 开发应用程序的开发者，简洁有效而又系统全面地介绍了如何基于 ChatGPT API 打造完整的对话系统；《LangChain for LLM Application Development》教程结合经典大模型开源框架 LangChain，介绍了如何基于 LangChain 框架开发具备实用功能、能力全面的应用程序，《LangChain Chat With Your Data》教程则在此基础上进一步介绍了如何使用 LangChain 架构结合个人私有数据开发个性化大模型应用；《Building Generative AI
source: https://aitutor.liduos.com/ref/ref.html

the ChatGPT API》教程面向想要基于 LLM 开发应用程序的开发者，简洁有效而又系统全面地介绍了如何基于 ChatGPT API 打造完整的对话系统；《LangChain for LLM Application Development》教程结合经典大模型开源框架 Lang

[WARNING][2024-10-24 09:13:24.434] base.py:916 [t:8703163072]: This key `disable_search` does not seem to be a parameter that the model `ERNIE-Speed-128K` will accept


除了LangChain，还有以下几个ChatGPT框架：

1. OpenAI的GPT-3框架：这是目前最流行的ChatGPT框架之一，由OpenAI开发和维护，提供各种基于AI的对话系统的能力，API可以进行即席试用和探索式迁移。开发人员可灵活地按需重建查询系统和下一代逻辑应用。GPT-3框架具有强大的自然语言处理能力，可实现高效的对话系统构建。2. Hugging Face的Transformers库：这是一个广泛使用的自然语言处理库，提供了许多预训练的语言模型，包括ChatGPT模型。Transformers库支持多种语言和任务，并且有丰富的文档和示例代码，方便开发者使用。Transformers库还提供了丰富的工具和功能，帮助开发者构建高效的对话系统。请注意，这些框架的使用方法各不相同。如果需要详细了解其特点和使用方式，请访问相关的官方网站获取最新的技术信息和支持文档。这些技术也在不断演进和更新中，因此建议查阅最新的官方文档或咨询专业人士以获取最新信息。

In [39]:
prompt.get_history_wo_role()

'langchain回调管理器如何使用\n除了langchain，还有哪些ChatGPT框架'